# Применение RNN для прогнозирования отношения в рецензиях на фильмы

## Подготовка данных

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

**Шаг 1:** создание набора данных.

In [ ]:
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices((df.values, target.values))
# инспектирование:
for ex in ds_raw.take(3):
  tf.print(ex[0].numpy()[0][:50], ex[1])

b'I went and saw this movie last night after being c' 1
b'Actor turned director Bill Paxton follows up his p' 1
b'As a recreational golfer with some knowledge of th' 1


In [ ]:
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

**Шаг 2:** нахождение уникальных лексем (слов)

In [ ]:
from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
  tokens = tokenizer.tokenize(example[0].numpy()[0])
  token_counts.update(tokens)

print('.размер словаря: ', len(token_counts))

.размер словаря:  87531


**Шаг 3:** кодирование уникальных лексем в виде целых чисел

In [ ]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'This is an example!'
print(encoder.encode(example_str))

[203, 53, 164, 846]


**Шаг 3-A:** определение функций для трансформации

In [ ]:
def encode(text_tensor, label):
  text = text_tensor.numpy()[0]
  encoded_text = encoder.encode(text)
  return encoded_text, label

**Шаг 3-Б:** помещение функции encode внутрь операции TensorFlow

In [ ]:
def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

Просмотр формы ряда образцов:

In [ ]:
tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
  print('Длина последовательности: ', example[0].shape)

Длина последовательности:  (70,)
Длина последовательности:  (256,)
Длина последовательности:  (203,)
Длина последовательности:  (150,)
Длина последовательности:  (74,)


## Разделение на пакеты

Извлечение небольшого поднабора:

In [ ]:
ds_subset = ds_train.take(8)
for example in ds_subset:
  print('Размер индивидуального элемента: ', example[0].shape)

Размер индивидуального элемента:  (193,)
Размер индивидуального элемента:  (182,)
Размер индивидуального элемента:  (288,)
Размер индивидуального элемента:  (57,)
Размер индивидуального элемента:  (256,)
Размер индивидуального элемента:  (96,)
Размер индивидуального элемента:  (27,)
Размер индивидуального элемента:  (345,)


Разделение поднабора на пакеты:

In [ ]:
ds_batched = ds_subset.padded_batch(4, padded_shapes=([-1], []))
for batch in ds_batched:
  print('Размерность пакета: ', batch[0].shape)

Размерность пакета:  (4, 288)
Размерность пакета:  (4, 345)


In [ ]:
train_data = ds_train.padded_batch(32, padded_shapes=([-1], []))
valid_data = ds_valid.padded_batch(32, padded_shapes=([-1], []))
test_data = ds_test.padded_batch(32, padded_shapes=([-1], []))

## Слои вложений для кодирования предложений

In [ ]:
from unicodedata import name
#example with Слои вложений
from tensorflow import keras
from keras import layers
from keras.layers import Embedding
model = tf.keras.Sequential()
model.add(Embedding(input_dim=100,
                    output_dim=6,
                    input_length=20,
                    name='embed-layer'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, 20, 6)             600       
                                                                 
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


## Построение модели

In [ ]:
embedding_dim = 20
vocab_size = len(token_counts) +2
tf.random.set_seed(1)
#построение модели
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'
    ),
    # помещаем слой LSTM внутрь оболочки Bidirectional 
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'
    ),

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])
bi_lstm_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed-layer (Embedding)     (None, None, 20)          1750660   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,802,501
Trainable params: 1,802,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from keras.engine.training import optimizer
#компиляция и обучение
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = bi_lstm_model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10
)

Epoch 1/10
625/625 [==============================] - 337s 534ms/step - loss: 0.5705 - accuracy: 0.7079 - val_loss: 0.4412 - val_accuracy: 0.8168
Epoch 2/10
625/625 [==============================] - 333s 533ms/step - loss: 0.3715 - accuracy: 0.8487 - val_loss: 0.5017 - val_accuracy: 0.8130
Epoch 3/10
625/625 [==============================] - 330s 528ms/step - loss: 0.2307 - accuracy: 0.9169 - val_loss: 0.4004 - val_accuracy: 0.8534
Epoch 4/10
625/625 [==============================] - 327s 523ms/step - loss: 0.1834 - accuracy: 0.9351 - val_loss: 0.4849 - val_accuracy: 0.8516
Epoch 5/10
625/625 [==============================] - 330s 528ms/step - loss: 0.0926 - accuracy: 0.9688 - val_loss: 0.5428 - val_accuracy: 0.8676
Epoch 6/10
625/625 [==============================] - 325s 520ms/step - loss: 0.0433 - accuracy: 0.9873 - val_loss: 0.6232 - val_accuracy: 0.8652
Epoch 7/10
625/625 [==============================] - 326s 522ms/step - loss: 0.0267 - accuracy: 0.9923 - val_loss: 0.6898 -

##Оценка на испытательных данных

In [ ]:
test_results=bi_lstm_model.evaluate(test_data)
print('{:.2f}%'.format(test_results[1]*100) )

782/782 [==============================] - 91s 117ms/step - loss: 0.7998 - accuracy: 0.8415
84.15%
